In [1]:
import pandas as pd
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

data_directory = Path('../data/processed')
try:
    df_final = pd.read_csv(data_directory / 'final_master_dataset_for_prediction.csv')
    print("✅ Tải dữ liệu đã xử lý thành công!")
    print(f"Kích thước dữ liệu: {df_final.shape}")
except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file {e.filename}. Vui lòng chạy notebook data-exploration trước.")
    exit()

✅ Tải dữ liệu đã xử lý thành công!
Kích thước dữ liệu: (3335, 31)


In [2]:
df_final = pd.get_dummies(df_final, columns=['Team_Tier'], prefix='TeamTier', drop_first=False)

In [3]:
# --- 2. Tách Features (X) và Target (y) ---
# Xác định cột mục tiêu (target)
TARGET = 'FinalPosition'

COLS_TO_DROP = [
    TARGET,
    'Year',
    'RaceName',
    'DriverNumber',
    'EventDate',
    'Points',
    'RoundNumber',
    'Status',
    'TeamName'
]

# Tạo X bằng cách loại bỏ các cột không cần thiết
# Dùng errors='ignore' để code không bị lỗi nếu một cột trong list không tồn tại
X = df_final.drop(columns=COLS_TO_DROP, errors='ignore')

# Tạo y từ cột target
y = df_final[TARGET]

# --- 3. Phân chia dữ liệu theo thời gian ---
# Xác định năm cuối cùng làm dữ liệu kiểm tra (test)
test_year = df_final['Year'].max()
print(f"Năm được chọn làm bộ dữ liệu test: {int(test_year)}")

# Tạo điều kiện lọc (boolean mask) cho tập train và test
train_mask = (df_final['Year'] < test_year)
test_mask = (df_final['Year'] == test_year)

# Áp dụng điều kiện lọc để tạo các tập dữ liệu
X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]


print(f"Huấn luyện trên {X_train.shape[0]} mẫu, bao gồm {X_train.shape[1]} features.")
print(f"Kiểm tra trên {X_test.shape[0]} mẫu, bao gồm {X_test.shape[1]} features.")

Năm được chọn làm bộ dữ liệu test: 2025
Huấn luyện trên 2976 mẫu, bao gồm 24 features.
Kiểm tra trên 359 mẫu, bao gồm 24 features.


In [4]:
# =============================================================================
# SECTION 3: HUẤN LUYỆN VÀ ĐÁNH GIÁ MÔ HÌNH
# =============================================================================
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    early_stopping_rounds=50,
    random_state=42
)

# Huấn luyện mô hình
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# Đánh giá mô hình
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f"✅ Huấn luyện hoàn tất! Sai số dự đoán trung bình (MAE): {mae:.2f} vị trí")

✅ Huấn luyện hoàn tất! Sai số dự đoán trung bình (MAE): 3.23 vị trí


In [5]:
import optuna

def objective(trial):
    """
    Hàm mục tiêu: thử các hyperparameters khác nhau và return MAE
    """
    # Định nghĩa search space
    params = {
        'objective': 'reg:squarederror',
        'random_state': 42,
        'early_stopping_rounds': 50,  # ✅ Đưa vào constructor thay vì fit()
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 3),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 10),
    }

    # Train model - early_stopping_rounds đã có trong params
    model = XGBRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )

    # Tính MAE
    preds = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds)

    return mae

print("✅ Objective function đã được định nghĩa!")

✅ Objective function đã được định nghĩa!


In [6]:
print("🔍 Bắt đầu tìm kiếm hyperparameters tốt nhất...")
print("⏱️  Quá trình này có thể mất 10-30 phút tùy thuộc vào n_trials")
print("=" * 70)

# Tạo study
study = optuna.create_study(
    direction='minimize',
    study_name='xgboost_f1_optimization',
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Chạy optimization
study.optimize(
    objective,
    n_trials=500,  # Thử 100 lần (có thể tăng lên 200-500)
    timeout=1800,   # 30 phút timeout
    show_progress_bar=True
)

print("\n✅ Hoàn thành!")
print("=" * 70)
print(f"\n🎯 Best MAE: {study.best_value:.4f}")
print(f"\n📋 Best Hyperparameters:")
for param, value in study.best_params.items():
    print(f"   • {param:20s}: {value}")

[I 2025-10-15 22:37:34,508] A new study created in memory with name: xgboost_f1_optimization


🔍 Bắt đầu tìm kiếm hyperparameters tốt nhất...
⏱️  Quá trình này có thể mất 10-30 phút tùy thuộc vào n_trials


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-10-15 22:37:34,654] Trial 0 finished with value: 3.383356586140178 and parameters: {'n_estimators': 812, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 5, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481, 'gamma': 0.17425083650459838, 'reg_alpha': 4.330880728874676, 'reg_lambda': 6.41003510568888}. Best is trial 0 with value: 3.383356586140178.
[I 2025-10-15 22:37:35,389] Trial 1 finished with value: 3.2898211704987337 and parameters: {'n_estimators': 1446, 'learning_rate': 0.010725209743171996, 'max_depth': 10, 'min_child_weight': 6, 'subsample': 0.6849356442713105, 'colsample_bytree': 0.6727299868828402, 'gamma': 0.5502135295603015, 'reg_alpha': 1.5212112147976886, 'reg_lambda': 5.72280788469014}. Best is trial 1 with value: 3.2898211704987337.
[I 2025-10-15 22:37:35,701] Trial 2 finished with value: 3.2463031180389743 and parameters: {'n_estimators': 921, 'learning_rate': 0.02692655251486473, 'max_depth': 7, 'min_child_weight':

In [15]:
print("🚀 Training model với hyperparameters tốt nhất...\n")

# Tạo params dict
best_params = study.best_params.copy()
best_params['objective'] = 'reg:squarederror'
best_params['random_state'] = 42
best_params['early_stopping_rounds'] = 50  # ✅ Thêm vào params

# Train - early_stopping_rounds đã có trong best_params
final_model = XGBRegressor(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

# Evaluate
final_predictions = final_model.predict(X_test)
final_mae = mean_absolute_error(y_test, final_predictions)

print("✅ Model đã được train xong!")
print(final_mae)

🚀 Training model với hyperparameters tốt nhất...

✅ Model đã được train xong!
3.0630652263636042


In [18]:
print("Bắt đầu huấn luyện mô hình với bộ feature đầy đủ...")

# 1. Khởi tạo mô hình với các tham số tốt nhất đã tìm được
# Đảm bảo các tham số này được tối ưu trên bộ feature đầy đủ

final_model = XGBRegressor(**best_params)

# 2. Huấn luyện mô hình trên bộ dữ liệu đầy đủ (X_train, không phải X_train_selected)
final_model.fit(X_train, y_train,
                eval_set=[(X_test, y_test)],
                verbose=False)

# 3. Đánh giá kết quả
final_predictions = final_model.predict(X_test)
final_mae = mean_absolute_error(y_test, final_predictions)

print("\n----------------------------------------------------")
print(f"✅ HOÀN TẤT!")
print(f"   => MAE với bộ feature đầy đủ: {final_mae:.4f} vị trí")
print("----------------------------------------------------")

Bắt đầu huấn luyện mô hình với bộ feature đầy đủ...

----------------------------------------------------
✅ HOÀN TẤT!
   => MAE với bộ feature đầy đủ: 3.0631 vị trí
----------------------------------------------------


In [19]:
import joblib
# --- LƯU MODEL VÀ DANH SÁCH FEATURES ---

# 1. Định nghĩa tên file để lưu
model_filename = '../models/f1_prediction_model.joblib'
features_filename = '../models/f1_model_features.joblib'

# 2. Lưu đối tượng model đã huấn luyện
joblib.dump(final_model, model_filename)

# 3. Lấy ra danh sách tên các feature và lưu lại
feature_list = X_train.columns.tolist()
joblib.dump(feature_list, features_filename)

print(f"✅ Model đã được lưu thành công vào file: '{model_filename}'")
print(f"✅ Danh sách {len(feature_list)} features đã được lưu vào file: '{features_filename}'")

✅ Model đã được lưu thành công vào file: 'f1_prediction_model.joblib'
✅ Danh sách 24 features đã được lưu vào file: 'f1_model_features.joblib'
